<a href="https://www.kaggle.com/code/janjibdev/titanic-3?scriptVersionId=103244954" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Separate target from predictors
y = df["Survived"]
X = df.drop("Survived",axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train_full,X_valid_full, y_train_full, y_valid_full = train_test_split(X,y,test_size = 0.2, random_state = 0) 

In [5]:
categorical_columns = ["Sex","Embarked"]
numerical_columns = ["Pclass","Age","SibSp","Parch","Fare"]

# list of columns used
column_needed = categorical_columns + numerical_columns

# assign useful data
X_train = X_train_full[column_needed].copy()
X_valid = X_valid_full[column_needed].copy()

In [6]:
from sklearn.impute import SimpleImputer
# preprocessing for numerical data (impute missing value)
missing_value_transformer = SimpleImputer(strategy = "most_frequent")

from sklearn.preprocessing import OneHotEncoder
# preprocessing for categorical data (one hot encode sex and embarked)
encoder = OneHotEncoder(handle_unknown="ignore")

from sklearn.compose import ColumnTransformer
# bundle preprocessing for data
preprocessor = ColumnTransformer(
        transformers= [
            ("num", missing_value_transformer, numerical_columns),
            ("cat", encoder, categorical_columns)
        ]
)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
def score(n_estimators):
    model = RandomForestClassifier(random_state = 0, max_depth = 3, n_estimators = n_estimators)
    my_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    my_pipeline.fit(X_train, y_train_full)
    preds = my_pipeline.predict(X_valid)
    score = mean_absolute_error(y_valid_full, preds)
    return score


In [8]:
n_list = [n for n in range(50,600,50)]
ans = {}
for i in n_list:
    ans[i] = score(i)
    
    
ans

{50: 0.18994413407821228,
 100: 0.18994413407821228,
 150: 0.19553072625698323,
 200: 0.19553072625698323,
 250: 0.2011173184357542,
 300: 0.2011173184357542,
 350: 0.2011173184357542,
 400: 0.2011173184357542,
 450: 0.2011173184357542,
 500: 0.2011173184357542,
 550: 0.19553072625698323}

In [9]:
test_set = pd.read_csv("/kaggle/input/titanic/test.csv")
test_set = test_set[column_needed].copy()
test_set

,Sex,Embarked,Pclass,Age,SibSp,Parch,Fare
0,male,Q,3,34.5,0,0,7.8292
1,female,S,3,47.0,1,0,7.0000
2,male,Q,2,62.0,0,0,9.6875
3,male,S,3,27.0,0,0,8.6625
4,female,S,3,22.0,1,1,12.2875
...,...,...,...,...,...,...,...
413,male,S,3,NaN,0,0,8.0500
414,female,C,1,39.0,0,0,108.9000
415,male,S,3,38.5,0,0,7.2500
416,male,S,3,NaN,0,0,8.0500


In [10]:
def submit(test_set = test_set):
    model = RandomForestClassifier(random_state = 0, max_depth = 3)
    my_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    my_pipeline.fit(X_train, y_train_full)
    preds = my_pipeline.predict(test_set)
    
    sub = {"PassengerId":[x for x in range(892,1310)],
                  "Survived": preds}

    submission = pd.DataFrame(sub)
    
    submission.to_csv("submission.csv",index = False)
    print("success")


    

In [11]:
submit()

success
